In [23]:
%pip install tensorflow==2.5.0 scikit-learn optuna

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.5.0
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import numpy as np

In [25]:
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [26]:
import tensorflow as tf

In [27]:
print(tf.__version__)
print(np.__version__)

2.18.0
2.2.1


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [29]:
def get_mlp_model(input_shape,hidden_layer_one,dropout_one,hidden_layer_two,dropout_two):
    model = Sequential()
    model.add(Dense(hidden_layer_one, activation='elu', input_shape=input_shape))
    if dropout_one !=0:
        model.add(Dropout(dropout_one))
    model.add(Dense(hidden_layer_two, activation='elu'))
    if dropout_two !=0:
        model.add(Dropout(dropout_two))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer="Adam")

    return model

In [30]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    rrse = np.sqrt(np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    mape = np.mean(np.abs((predicted - trues) / trues)) * 100
    r2 = max(0, 1 - np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    # Calculez les autres mesures d'erreur ici
    return rmse, corr, mae, rae, rrse, mape, r2

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import optuna
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.neural_network import MLPRegressor
from tensorflow.keras.callbacks import ModelCheckpoint

/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

W0000 00:00:1740078645.255337   27266 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

# We start by loading the pre-processed data

In [33]:
order_history_data=pd.read_csv('../data/final_data.csv')
order_history_data.head()

,Inflation,Month,Year,pmi,PartNo,Description,Quantity,Sales Price,Tractor Sales
0,7.59,1,2020,250,0503BA0290N-RK,ALFA HUB,85,2118.60,59367
1,7.59,1,2020,250,0503BA0290N,Front Wheel Hub,318,3536.35,59367
2,7.59,1,2020,250,0502BA1050N,Alfa Rear Wheel Hub LH,209,1383.36,59367
3,7.59,1,2020,250,0502BA1060N,Alfa Rear Wheel Hub RH,119,4849.50,59367
4,7.59,1,2020,250,0602AAB00260N,Brake Drum ALFA,134,2462.26,59367


# Deep learning model optimization

In [34]:
X = order_history_data.copy()
y = order_history_data['Quantity']

X_encoded = pd.get_dummies(X, columns=['Description', 'PartNo'])

X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

We're going to use mlp. To start with, we'll create an mlp with minimal parameters to get an idea of performance without model optimization. Then we'll optmize our model using Bayesian optimization.

In [35]:

best_mlp = MLPRegressor(hidden_layer_sizes=(50), activation='relu', alpha=0.008724528119026307, learning_rate='constant')

# Train the model on the training data
best_mlp.fit(X_train, y_train)

# Make predictions on the validation set with the trained model
y_val_pred = best_mlp.predict(X_val)

# Evaluate the performance of the model on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = best_mlp.score(X_val, y_val)

# Make predictions on the test set with the trained model
y_test_pred = best_mlp.predict(X_test)

# Evaluate the performance of the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = best_mlp.score(X_test, y_test)

print("Validation Set:")
print("MSE (Mean Squared Error) on the validation set:", mse_val)
print("R-squared (Coefficient of Determination) on the validation set:", r2_val)

print("\nTest Set:")
print("MSE (Mean Squared Error) on the test set:", mse_test)
print("R-squared (Coefficient of Determination) on the test set:", r2_test)



Validation Set:
MSE (Mean Squared Error) on the validation set: 5.756574845217267
R-squared (Coefficient of Determination) on the validation set: 0.9997325597669764

Test Set:
MSE (Mean Squared Error) on the test set: 5.774604008953301
R-squared (Coefficient of Determination) on the test set: 0.9997429925181438


Using Bayesian optimization to find the best hyperparameters

In [ ]:

# Objective function for Optuna optimization
def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_layer_one = trial.suggest_categorical('hidden_layer_one', [32, 64, 128, 256,512])
    hidden_layer_two = trial.suggest_categorical('hidden_layer_two', [32, 64, 128, 256,512])
    dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
    dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128,256,512])
    epochs = trial.suggest_categorical('epochs', [200])



    model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)


    # Define the ModelCheckpoint callback
    checkpoint_filepath = 'model_checkpoint.h5'
    model_checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=0
    )

    # Train the model on the training data with validation data and checkpoint callback
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0
                        , validation_data=(X_val, y_val),
                        callbacks=[model_checkpoint])

    # Load the best weights from the saved checkpoint
    model.load_weights(checkpoint_filepath)

    # Evaluate the performance of the model on the test set
    y_test_pred = model.predict(X_test)

    # Calculate performance metrics
    mse_test = mean_squared_error(y_test, y_test_pred)
    # rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

    return mse_test

# Configure Optuna to use the GPU for exhaustive searches
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=1)  # Use n_jobs=1 to avoid parallelism problems on GPU

# Show best hyperparameters found
print("Best hyperparameters:")
print(study.best_params)
print("Best MSE:", study.best_value)


[I 2025-02-21 00:41:21,935] A new study created in memory with name: no-name-fcb6e277-3284-43dd-8b37-2fac638b6139
/tmp/ipykernel_27266/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_27266/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regu

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


[I 2025-02-21 00:44:37,676] Trial 0 finished with value: 118.40306091308594 and parameters: {'hidden_layer_one': 512, 'hidden_layer_two': 256, 'dropout_one': 0.4475138145660727, 'dropout_two': 0.3386159540707072, 'batch_size': 32, 'epochs': 200}. Best is trial 0 with value: 118.40306091308594.
/tmp/ipykernel_27266/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_27266/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


[I 2025-02-21 00:45:58,097] Trial 1 finished with value: 197.69679260253906 and parameters: {'hidden_layer_one': 512, 'hidden_layer_two': 32, 'dropout_one': 0.41057479590296314, 'dropout_two': 0.24840602154990943, 'batch_size': 64, 'epochs': 200}. Best is trial 0 with value: 118.40306091308594.
/tmp/ipykernel_27266/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_27266/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input

...

[I 2025-02-21 00:47:09,748] Trial 2 finished with value: 118.118896484375 and parameters: {'hidden_layer_one': 128, 'hidden_layer_two': 32, 'dropout_one': 0.3991411689488171, 'dropout_two': 0.4981836372471809, 'batch_size': 32, 'epochs': 200}. Best is trial 2 with value: 118.118896484375.
/tmp/ipykernel_27266/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_27266/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-02-21 00:48:13,326] Trial 3 finished with value: 30.944469451904297 and parameters: {'hidden_layer_one': 256, 'hidden_layer_two': 128, 'dropout_one': 0.01321910955152883, 'dropout_two': 0.17256904949441015, 'batch_size': 64, 'epochs': 200}. Best is trial 3 with value: 30.944469451904297.
/tmp/ipykernel_27266/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_27266/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `inpu

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


[I 2025-02-21 00:52:40,450] Trial 4 finished with value: 317.3367919921875 and parameters: {'hidden_layer_one': 512, 'hidden_layer_two': 512, 'dropout_one': 0.46247231345800294, 'dropout_two': 0.445366477031361, 'batch_size': 32, 'epochs': 200}. Best is trial 3 with value: 30.944469451904297.
/tmp/ipykernel_27266/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_27266/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_s

Using Grid Search

In [ ]:

r2=0

for hidden_layer_one in [25,50,75,100,125,150,175,200,225,350,300,350,400]:
  for hidden_layer_two in [325,50,75,100,125,150,175,200,225,350,300,350,400]:
      for dropout_one in [0,0.1,0.2,0.3,0.4,0.5]:
        for dropout_two in [0,0.1,0.2,0.3,0.4,0.5]:
          # for batch_size in [32,64,128,256]:


            model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

            # Define the ModelCheckpoint callback
            checkpoint_filepath = 'model_checkpoint.h5'
            model_checkpoint = ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_best_only=True,
                monitor='val_loss',
                mode='min',
                verbose=0
            )

            # Train the model on the training data with validation data and checkpoint callback
            history = model.fit(X_train, y_train, epochs=300, batch_size=256,
                                verbose=0, validation_data=(X_val, y_val),
                                callbacks=[model_checkpoint])

            # Load the best weights from the saved checkpoint
            model.load_weights(checkpoint_filepath)

            # Evaluate the performance of the model on the test set
            y_test_pred = model.predict(X_test)

            # Calculate performance metrics
            mse_test = mean_squared_error(y_test, y_test_pred)
            rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

            if r2_2>r2:
              r2=r2_2
              print("{},{},{},{}".format(hidden_layer_one,hidden_layer_two,dropout_one,dropout_two) )
              print("RMSE:", rmse2)
              print("Corrélation:", corr2)
              print("MAE:", mae2)
              print("RAE:", rae2)
              print("RRSE:", rrse2)
              print("MAPE:", mape2)
              print("R2:", r2_2)
              print("MSE:", mse_test)
              print("----------------------------------")



Training model with best hyperparamters

In [ ]:
# best parameters

hidden_layer_one=50
hidden_layer_two=100
dropout_one=0
dropout_two=0


model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

# Define the ModelCheckpoint callback
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint = ModelCheckpoint(
filepath=checkpoint_filepath,
save_best_only=True,
monitor='val_loss',
mode='min',
verbose=0
)

# Train the model on the training data with validation data and checkpoint callback
history = model.fit(X_train, y_train, epochs=300, batch_size=512,
            verbose=2, validation_data=(X_val, y_val),
            callbacks=[model_checkpoint])

# Load the best weights from the saved checkpoint
model.load_weights(checkpoint_filepath)

# Evaluate the performance of the model on the test set
y_test_pred = model.predict(X_test)

# Calculate performance metrics
rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

print("RMSE:", rmse2)
print("Corrélation:", corr2)
print("MAE:", mae2)
print("RAE:", rae2)
print("RRSE:", rrse2)
print("MAPE:", mape2)
print("R2:", r2_2)
print("----------------------------------")

Save the weights of the best model for later loading

In [ ]:
# Save the best model in checkpoint only if it is better
# model.save_weights('../checkpoint/base_model_checkpoint.h5')